# Preparation of Three Tank Simulation Datasets
Execute the following lines of code for the generation of the datasets. <br>
The Simulation will output a normal sequence of the tank process and different datasets including anomalies. <br>
The anomalies will be a fault in valve v12, v23, v3 and a fault in the inflow q1.<br>
How the simulation is build and which parameters are included can be read within the paper of Steude et. al (https://doi.org/10.1016/j.ifacol.2022.07.099)

In [1]:
cd ..

/home/jovyan/etfa_discret2deepdive


In [2]:
from typing import Tuple
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from tqdm import tqdm

In [3]:
class ThreeTankSimulation:
    """Simulates the three tank system.
    The system is simulated using the scipy odeint function.
    """
    def __init__(self, tank_1_lvl=0, tank_2_lvl=0, tank_3_lvl=0, seed=42):
        self.tank_levels = np.array([tank_1_lvl, tank_2_lvl, tank_3_lvl])
        self.seed = seed
        self.state_df = pd.DataFrame(columns=["q1", "q3", "kv1", "kv2", "kv3", "duration"])

    def add_state(self, q1: float, q3: float, kv1: float, kv2: float, kv3: float, duration: int, name=None) -> None:
        """Add a state to the state dataframe.
        A state consists of specific settings to the system's parameters.
        Args:
            q1 (float): inflow tank 1
            q3 (float): inflow tank 3
            kv1 (float): coefficient of the valve between tank 1 and 2
            kv2 (float): coefficient of the valve between tank 2 and 3
            kv3 (float): coefficient of the outgoing valve on tank 3
            duration (int): number of time steps of the state
            name (string): the name of the state
        """
        if name is not None:
            self.state_df.loc[name] = [q1, q3, kv1, kv2, kv3, duration]
        else:
            self.state_df.append(dict(q1=q1, q3=q3, kv1=kv1, kv2=kv2, kv3=kv3, duration=duration),
                                 ignore_index=True)

    @staticmethod
    def _system_dynamics_function(x, t, q1, q3, kv1, kv2, kv3):
        # ensure non-negative tank levels
        x1, x2, x3 = x * (x > 0)
        # ODE
        dh1_dt = q1 - kv1 * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2))
        dh2_dt = kv1 * np.sign(x1 - x2) * np.sqrt(np.abs(x1 - x2)) \
                 - kv2 * np.sign(x2 - x3) * np.sqrt(np.abs(x2 - x3))
        dh3_dt = q3 + kv2 * np.sign(x2 - x3) * np.sqrt(np.abs(x2 - x3)) - kv3 * np.sqrt(x3)

        return dh1_dt, dh2_dt, dh3_dt

    def _compute_section(self, duration: int = 10, x0: np.array = np.array([30, 10, 50]),
                         kv1: float = 1, kv2: float = 1, kv3: float = 1,
                         q1: float = 1, q3: float = 1):
        t = np.array(range(duration))
        y = odeint(self._system_dynamics_function, x0, t, (q1, q3, kv1, kv2, kv3))
        # non-negativity
        y = y * (y > 0)
        y_stop = y[-1, :]
        return y, y_stop

    @staticmethod
    def _duplicate_row(row, factor):
        return pd.concat([row.copy()] * factor, axis = 1)

    def _configuration_seq(self, cycle: list, nb_of_cycles: int,
                           sd_q: float, sd_kv: float, sd_dur: float,
                           leaky: bool, periodic_inflow: bool) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """generates configuration dataframes
        The configuration dataframe describes the state at every time step.
        Outputs original state configuration and configuration with noise.
        """
        # generate cycle of states
        seq = list()
        for i in range(nb_of_cycles):
            for state in cycle:
                if type(state) is str:
                    seq.append(self.state_df.loc[state])
                else:
                    seq.append((self.state_df.iloc[state, :]))
        seq_df = pd.concat(seq, axis=1).T.astype({"duration": int})
        seq_len = seq_df.shape[0]

        # add periodic inflow
        if periodic_inflow:
            amplitude = 0.5 * seq_df["q1"].max()
            wave = amplitude * np.cos(np.linspace(np.pi, 5*np.pi, 2000))

            q1_mask = seq_df["q1"] > 0
            q3_mask = seq_df["q3"] > 0
            
            seq_df.loc[q1_mask, "q1"] += wave[:q1_mask.sum()]
            seq_df.loc[q3_mask, "q3"] += wave[:q3_mask.sum()]

        # add noise
        np.random.seed(self.seed)
        seq_df_noise = seq_df.copy()
        if sd_q is not None:
            q_noise = np.random.normal(0, sd_q, 2 * seq_len)
            seq_df_noise["q1"] = seq_df["q1"] + q_noise[:seq_len]
            seq_df_noise["q3"] = seq_df["q3"] + q_noise[seq_len:]
            if not leaky:
                seq_df_noise["q1"].where(seq_df["q1"] > 0, other=0, inplace=True)  # no leaky inflow
                seq_df_noise["q3"].where(seq_df["q3"] > 0, other=0, inplace=True)  # (set back to 0 if no inflow)
        if sd_kv is not None:
            kv_noise = np.random.normal(0, sd_kv, 3 * seq_len)
            seq_df_noise["kv1"] = seq_df["kv1"] + kv_noise[:seq_len]
            seq_df_noise["kv2"] = seq_df["kv2"] + kv_noise[seq_len:2*seq_len]
            seq_df_noise["kv3"] = seq_df["kv3"] + kv_noise[2*seq_len:]
            if not leaky:
                seq_df_noise["kv1"].where(seq_df["kv1"] > 0, other=0, inplace=True)  # no leaky valve
                seq_df_noise["kv2"].where(seq_df["kv2"] > 0, other=0, inplace=True)
                seq_df_noise["kv3"].where(seq_df["kv3"] > 0, other=0, inplace=True)
        if sd_dur is not None:
            dur_noise = np.random.normal(0, sd_dur, seq_len)
            seq_df_noise["duration"] = round(seq_df["duration"] + dur_noise).astype(int)
        # no negative inflow etc.
        seq_df = seq_df.where(seq_df >= 0, 0)
        seq_df_noise = seq_df_noise.where(seq_df_noise >= 0, 0) 

        return seq_df, seq_df_noise

    @staticmethod
    def _export_config(seq_df, seq_df_noise, export_path):
        """exports state configuration dataframe
        Transforms the dataframe so that the state at every time step is exported.
        """
        seq0 = list()
        seq0_noise = list()
        for (_, row), (_, row_noise) in zip(seq_df.iterrows(), seq_df_noise.iterrows()):
            duration = int(row_noise.duration)  # actual duration
            seq0 += [row] * duration
            seq0_noise += [row_noise] * duration
        seq0_df = pd.concat(seq0_noise, axis=1).T
        seq0_df.to_csv(f"{export_path[:-4]}_config.csv", index=False)

    def simulate(self, cycle_list: list, nb_of_cycles_list: list,
                 sd_q: float = None, sd_kv: float = None, sd_dur: float = None, sd_white_noise: float = None, 
                 leaky: bool = False, periodic_inflow = False,
                 export_path: str = None) -> np.array:
        """Simulates the dynamics in the three-tank system
        Args:
            cycle (list): sequence of states that compose a typical cycle.
                          Either list of integers or list of state names.
            nb_of_cycles (int): number of successive cycles to simulate
            sd_q (float): if set, white noise with this standard deviation is added to the inflow
            sd_kv (float): if set, white noise with this standard deviation is added to the valve coefficients
            sd_dur (float): if set, white noise with this standard deviation is added to the duration
            leaky (bool): if true, add noise on closed valves or stopped inflow
            periodic_inflow (bool): if true, add periodic variation to the inflow
            export_path (str): if set, save simulation data at export path
        """
        seq_denoised_list = list()
        seq_list = list()
        cycle_names = []  # New list to store cycle names               
        for cycle, nb_of_cycles in zip(cycle_list, nb_of_cycles_list):
            seq_denoised, seq = self._configuration_seq(cycle, nb_of_cycles, sd_q, sd_kv, sd_dur, leaky, periodic_inflow)
            seq_denoised_list.append(seq_denoised)
            seq_list.append(seq)
            cycle_names.extend([cycle] * len(seq_denoised))  # Repeat cycle name for each row
        
        seq_denoised = pd.concat(seq_denoised_list, axis=0)
        seq = pd.concat(seq_list, axis=0)
        
        y_ls = []
        cycles_ls = []
        y_stop = self.tank_levels
        # for config in tqdm(seq.itertuples(), total=len(seq)):
        for config, cycle_name in tqdm(zip(seq.itertuples(), cycle_names), total=len(seq)):
            y, y_stop = self._compute_section(duration=config.duration, x0=y_stop,
                                            kv1=config.kv1, kv2=config.kv2, kv3=config.kv3,
                                            q1=config.q1, q3=config.q3)
            
            y_ls.append(y)
            cycles = [config.Index] * config.duration
            cycles_ls.append(cycles)
        y_out = np.concatenate(y_ls, axis=0)
        cycles_ls = np.concatenate(cycles_ls, axis=0)

        if sd_white_noise is not None:
            np.random.seed(self.seed)
            y_out += np.random.normal(0, sd_white_noise, y_out.shape)

        if export_path is not None:
            y_df = pd.DataFrame(y_out, columns=['h1', 'h2', 'h3'])
            y_df['cycle_name'] = cycles_ls  # Add cycle_name column
            y_df.iloc[3500:,:].reset_index(drop=True).to_csv(export_path, index=False)
            self._export_config(seq_denoised, seq, export_path)

        return y_out


### Creation of a high repetition of first cycle of cycle list with (15, 1, 15)


In [4]:
tank_1_lvl=7
tank_2_lvl=3
tank_3_lvl=0
q = 0.1 # inflow
kv = 0.1 # valve coefficient
duration = 50
duration_anomaly = 50

system = ThreeTankSimulation(tank_1_lvl=tank_1_lvl, tank_2_lvl=tank_2_lvl, tank_3_lvl=tank_3_lvl)
system.add_state(q1=q, q3=0, kv1=0,  kv2=0,  kv3=0,  duration=duration, name="q1")
system.add_state(q1=q, q3=0, kv1=0,  kv2=0,  kv3=0,  duration=duration, name="q1_2")
system.add_state(q1=0, q3=0, kv1=kv, kv2=0,  kv3=0,  duration=duration, name="v12")
system.add_state(q1=0, q3=0, kv1=kv, kv2=0,  kv3=0,  duration=duration, name="v12_2")
system.add_state(q1=0, q3=0, kv1=0,  kv2=kv, kv3=0,  duration=duration, name="v23")
system.add_state(q1=0, q3=0, kv1=0,  kv2=kv, kv3=0,  duration=duration, name="v23_2")
system.add_state(q1=0, q3=0, kv1=0,  kv2=0,  kv3=kv, duration=duration, name="v3")
system.add_state(q1=0, q3=0, kv1=0,  kv2=0,  kv3=0,  duration=duration, name="rest")
system.add_state(q1=q, q3=0, kv1=0,  kv2=0,  kv3=kv, duration=duration, name="q1+v3")
system.add_state(q1=0, q3=0, kv1=kv, kv2=kv, kv3=0,  duration=duration, name="v12+v23")
system.add_state(q1=0, q3=q, kv1=0,  kv2=0,  kv3=0,  duration=duration, name="q3")

# faulty states
system.add_state(q1=0, q3=0, kv1=kv * 7, kv2=0,  kv3=0,  duration=duration_anomaly, name="v12-faulty")
system.add_state(q1=0, q3=0, kv1=0,  kv2=kv * 7, kv3=0,  duration=duration_anomaly, name="v23-faulty")
system.add_state(q1=0, q3=0, kv1=0,  kv2=0,  kv3=kv * 0.01, duration=duration_anomaly, name="v3-faulty")
system.add_state(q1=1, q3=0, kv1=0,  kv2=0,  kv3=0, duration=duration_anomaly, name="q1-faulty")
system.add_state(q1=0, q3=0, kv1=0,  kv2=0,  kv3=0,  duration=duration_anomaly, name="rest_faulty")
system.add_state(q1=q, q3=0, kv1=0,  kv2=0,  kv3=kv, duration=duration_anomaly, name="q1+v3_faulty")
system.add_state(q1=q, q3=0, kv1=0, kv2=kv, kv3=0, duration=duration_anomaly, name='q1+v23_faulty')
system.add_state(q1=0, q3=0, kv1=kv, kv2=0,  kv3=kv,  duration=duration_anomaly, name="v12+v3_faulty")
system.add_state(q1=0, q3=0, kv1=kv, kv2=kv, kv3=0,  duration=duration_anomaly, name="v12+v23_faulty")

# sd_noise = 0.001
# sd_q = 0.001  # for one scenario, we add noise on the inflow
sd_noise = 0.1
sd_q = 0.05
sd_dur = duration * 0.1  # for one scenario, we add noise on the duration

In [5]:
# Simulation of normal system states
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/norm.csv"
)

100%|██████████| 140/140 [00:06<00:00, 22.93it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.2760846 , 10.41790394,  0.82922563],
       [13.24212143, 10.33615308,  0.82147137],
       [13.43363554, 10.40107797,  0.37206751]])

In [6]:
# Simulation of an anomaly in q1 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1-faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/q1_50s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 27.36it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.64128699, 10.73967233,  0.91808806],
       [13.60732381, 10.65792147,  0.90537835],
       [13.79883792, 10.72284636,  0.45101904]])

In [7]:
# Simulation of an anomaly in v12 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12-faulty", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v12_50s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 24.08it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.27608458, 10.41790396,  0.82922562],
       [13.2421214 , 10.33615311,  0.82147136],
       [13.43363551, 10.401078  ,  0.3720675 ]])

In [8]:
# Simulation of an anomaly in v23 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23-faulty", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v23_50s_faulty.csv"
)

 21%|██▏       | 30/140 [00:01<00:03, 29.94it/s]/tmp/ipykernel_47596/892323876.py:44: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  y = odeint(self._system_dynamics_function, x0, t, (q1, q3, kv1, kv2, kv3))
100%|██████████| 140/140 [00:07<00:00, 18.33it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.29385432, 10.43353636,  0.83346009],
       [13.25989115, 10.35178551,  0.82546332],
       [13.45140526, 10.4167104 ,  0.37581695]])

In [9]:
# Simulation of an anomaly in v3 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3-faulty"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v3_50s_faulty.csv"
)

100%|██████████| 140/140 [00:06<00:00, 20.91it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.33779887, 10.472206  ,  0.84397157],
       [13.3038357 , 10.39045515,  0.83537569],
       [13.49534981, 10.45538004,  0.38513021]])

### Changing anomaly duration to 1sec with cycle repettitions (15, 1, 15)

In [10]:
duration_anomaly = 1
sd_noise = 0.1
sd_q = 0.05  # for one scenario, we add noise on the inflow
sd_dur = duration * 0.1  # for one scenario, we add noise on the duration

In [11]:
# Simulation of an anomaly in q1 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1-faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/q1short1s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 27.70it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.64128699, 10.73967233,  0.91808806],
       [13.60732381, 10.65792147,  0.90537835],
       [13.79883792, 10.72284636,  0.45101904]])

In [12]:
# Simulation of an anomaly in v12 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12-faulty", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v12short1s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 23.84it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.27608458, 10.41790396,  0.82922562],
       [13.2421214 , 10.33615311,  0.82147136],
       [13.43363551, 10.401078  ,  0.3720675 ]])

In [13]:
# Simulation of an anomaly in v23 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23-faulty", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v23short1s_faulty.csv"
)

 21%|██▏       | 30/140 [00:01<00:03, 29.83it/s]/tmp/ipykernel_47596/892323876.py:44: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  y = odeint(self._system_dynamics_function, x0, t, (q1, q3, kv1, kv2, kv3))
100%|██████████| 140/140 [00:07<00:00, 18.23it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.29385432, 10.43353636,  0.83346009],
       [13.25989115, 10.35178551,  0.82546332],
       [13.45140526, 10.4167104 ,  0.37581695]])

In [14]:
# Simulation of an anomaly in v3 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3-faulty"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v3short1s_faulty.csv"
)

100%|██████████| 140/140 [00:06<00:00, 20.83it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.33779887, 10.472206  ,  0.84397157],
       [13.3038357 , 10.39045515,  0.83537569],
       [13.49534981, 10.45538004,  0.38513021]])

### Changing anomaly duration to 100sec with cycle repettitions (15, 1, 15)

In [15]:
duration_anomaly = 100
sd_noise = 0.1
sd_q = 0.05  # for one scenario, we add noise on the inflow
sd_dur = duration * 0.1  # for one scenario, we add noise on the duration

In [16]:
# Simulation of an anomaly in q1 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1-faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/q1_100s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 27.50it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.64128699, 10.73967233,  0.91808806],
       [13.60732381, 10.65792147,  0.90537835],
       [13.79883792, 10.72284636,  0.45101904]])

In [17]:
# Simulation of an anomaly in v12 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12-faulty", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v12_100s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 23.88it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.27608458, 10.41790396,  0.82922562],
       [13.2421214 , 10.33615311,  0.82147136],
       [13.43363551, 10.401078  ,  0.3720675 ]])

In [18]:
# Simulation of an anomaly in v23 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23-faulty", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v23_100s_faulty.csv"
)

 21%|██▏       | 30/140 [00:01<00:03, 30.10it/s]/tmp/ipykernel_47596/892323876.py:44: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  y = odeint(self._system_dynamics_function, x0, t, (q1, q3, kv1, kv2, kv3))
100%|██████████| 140/140 [00:07<00:00, 18.32it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.29385432, 10.43353636,  0.83346009],
       [13.25989115, 10.35178551,  0.82546332],
       [13.45140526, 10.4167104 ,  0.37581695]])

In [19]:
# Simulation of an anomaly in v3 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3-faulty"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v3_100s_faulty.csv"
)

100%|██████████| 140/140 [00:06<00:00, 21.12it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.33779887, 10.472206  ,  0.84397157],
       [13.3038357 , 10.39045515,  0.83537569],
       [13.49534981, 10.45538004,  0.38513021]])

In [20]:
# Simulation of an anomaly in rest state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "rest_faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/rest_100s_faulty.csv"
)

100%|██████████| 140/140 [00:06<00:00, 22.16it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.24991314, 10.39488487,  0.82300593],
       [13.21594996, 10.31313402,  0.8156091 ],
       [13.40746407, 10.37805891,  0.36656268]])

In [21]:
# Simulation of an anomaly in v12 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v12+v23_faulty", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v12v23_100s_faulty.csv"
)

100%|██████████| 140/140 [00:06<00:00, 22.81it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.2708959 , 10.41333979,  0.82799092],
       [13.23693272, 10.33158893,  0.8203075 ],
       [13.42844683, 10.39651382,  0.37097448]])

In [22]:
# Simulation of an anomaly in v23 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1+v3_faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/q1v3_100s_faulty.csv"
)

100%|██████████| 140/140 [00:05<00:00, 23.82it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.2526309 , 10.39727499,  0.82365087],
       [13.21866772, 10.31552414,  0.81621691],
       [13.41018183, 10.38044903,  0.36713336]])

In [23]:
# Simulation of an anomaly in v3 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1+v23_faulty", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/q1v23_100s_faulty.csv"
)



100%|██████████| 140/140 [00:06<00:00, 21.71it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.2760846 , 10.41790395,  0.82922562],
       [13.24212143, 10.3361531 ,  0.82147136],
       [13.43363554, 10.40107799,  0.3720675 ]])

In [24]:
# Simulation of an anomaly in v3 state
system.simulate(cycle_list=[
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"],
    ["q1", "v12", "v23", "q1", "v12+v3_faulty", "v23"],
    ["q1", "v12", "v23", "q1", "v12", "v23", "v3"]
    ],
    nb_of_cycles_list=[4, 1, 15],
    sd_q=None, sd_kv=None, sd_dur=None, sd_white_noise=sd_noise,
    periodic_inflow=False,
    export_path="preprocessed_data/tank_simulation/v12v3_100s_faulty.csv"
)

100%|██████████| 139/139 [00:07<00:00, 19.23it/s]


array([[ 7.04967142,  2.98617357,  0.06476885],
       [ 7.25230299,  2.97658466, -0.0234137 ],
       [ 7.35792128,  3.07674347, -0.04694744],
       ...,
       [13.42722449, 10.40265299,  0.89894223],
       [13.49292929, 10.47292365,  0.80127504],
       [13.23333813, 10.27847703,  0.65068632]])